<a href="https://colab.research.google.com/github/arjasc5231/Lingometer/blob/speaker_verification/speaker_verification/batcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/arjasc5231/Lingometer/blob/speaker_verification/speaker_verificaiton/batcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os
import pickle
import random

import import_ipynb
from constants import NUM_FBANK, HOP_LENGTH, NUM_PER_SPEAKER, NUM_FRAME

importing Jupyter notebook from constants.ipynb


In [ ]:
class naive_batcher:
    def __init__(self, input_dir, output_dir):
        self.name = 'naive_batcher'
        self.input_dir = input_dir
        self.output_dir = output_dir
        
        if len(os.listdir(self.output_dir))==0: self.generate()
        else: print('load exist data')
        self.filename = os.listdir(self.output_dir)[0]
        self.num_speaker = int(self.filename.split('_')[-4])
    
    def generate(self, num_frame=NUM_FRAME, num_per_speaker=NUM_PER_SPEAKER):
        #if len(os.listdir(self.output_dir))!=0: error()
        X = []
        Y = []
        speaker_id = 0        
        
        for speaker in os.listdir(self.input_dir):
            X_speaker = []
            for utt in os.listdir(self.input_dir+'/'+speaker):
                spec = np.load(self.input_dir+'/'+speaker+'/'+utt)
                for i in range(0,spec.shape[0]-num_frame+1,num_frame): X_speaker.append(spec[i:i+num_frame,:])
                if len(X_speaker)>=num_per_speaker: break
            if len(X_speaker)<num_per_speaker: continue
            X.extend(X_speaker[:num_per_speaker])
            Y.extend([speaker_id]*num_per_speaker)
            speaker_id+=1
        X = np.array(X)
        Y = np.array(Y)
        
        # shuffle utterances
        shuffle_idx = np.arange(X.shape[0])
        np.random.shuffle(shuffle_idx)
        X = X[shuffle_idx]
        Y = Y[shuffle_idx]
        
        # split train/test
        split_idx = int(X.shape[0]*0.8)
        X_train, X_test, Y_train, Y_test = X[:split_idx], X[split_idx:], Y[:split_idx], Y[split_idx:]
        
        # print information
        print()
        print('generate train/test dataset')
        print('shape:', X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)
        
        # save
        XY = (X_train, X_test, Y_train, Y_test)
        with open(self.output_dir+'/'+f"{speaker_id}_{NUM_PER_SPEAKER}_{NUM_FRAME}_{NUM_FBANK}.npy",'wb') as f:
            pickle.dump(XY,f)
        
    def load_data(self):
        with open(self.output_dir+'/'+self.filename,"rb") as f:
            return pickle.load(f)

NameError: name 'NUM_FRAME' is not defined

In [ ]:
class simMat_batcher:
    # 원래 논문(코드)에서는 batch의 크기를 고정하고 대신 batch마다 시간축의 길이를 유동적으로 변할 수 있게 모델링했다.
    # 하지만 그렇게 하려면 모델의 구조를 바꿔야 하는걸... 시간 다르게는 나중에 구현하기
    def __init__(self, input_dir, output_dir):
        self.name = 'simMat_batcher'
        self.input_dir = input_dir
        self.output_dir = output_dir
        
        if len(os.listdir(self.output_dir))==0: self.generate()
        else: print('load exist data')
        self.filename = os.listdir(self.output_dir)[0]
        self.num_speaker = int(self.filename.split('_')[0]) # num_speaker는 이 batcher에서 의미없다. 일단 모델을 만들기 위해 설정
    
    # num_speaker_per_batch : 배치당 화자 수. 논문=64 코드=4. 일단은 5개 발화로 등록한다 가정하고 5로 설정.
    # num_utt_per_speaker : 배치당 화자마다 발화 수. 논문=10 코드=5
    # num_batch : 총 몇개의 배치(사실상 에폭)을 학습시킬 것인가. 코드=100000. 일단은 테스트만 해보려고 작게 1000으로 설정.
    def generate(self, num_frame=NUM_FRAME, num_speaker_per_batch=5, num_utt_per_speaker=5, num_batch=1000):
        
        specs_of_speaker = []
        for speaker in os.listdir(self.input_dir):
            X_speaker = []
            for utt in os.listdir(self.input_dir+'/'+speaker):
                spec = np.load(self.input_dir+'/'+speaker+'/'+utt)
                for i in range(0,spec.shape[0]-NUM_FRAME+1,NUM_FRAME): X_speaker.append(spec[i:i+NUM_FRAME,:])
                if len(X_speaker)>=NUM_PER_SPEAKER: break
            if len(X_speaker)<NUM_PER_SPEAKER: continue
            specs_of_speaker.append(np.array(X_speaker[:NUM_PER_SPEAKER]))
        specs_of_speaker = np.array(specs_of_speaker)
        num_speaker = len(specs_of_speaker)
        print('number of speakers who has more than minimun:', num_speaker)

        # sample utterances for train
        X = []
        for i in range(num_batch):
            batch = []
            speakers = np.random.choice(num_speaker, num_speaker_per_batch, replace=False)
            for speaker in speakers:
                utts_idx = np.random.choice(NUM_PER_SPEAKER, num_utt_per_speaker, replace=False)
                batch.append(specs_of_speaker[speaker][utts_idx])
            batch = np.concatenate(batch)
            X.append(batch)
        X = np.array(X)
        print('total generated shape:',X.shape)

        Y = np.zeros(X.shape[:2]) # dummy
        print('dummy Y shape:',Y.shape)

        # save
        XY = (X, Y)
        with open(self.output_dir+'/'+f"{num_speaker}_{NUM_PER_SPEAKER}_{NUM_FRAME}_{NUM_FBANK}_{num_speaker_per_batch}_{num_utt_per_speaker}_{num_batch}.npy",'wb') as f:
            pickle.dump(XY,f)
        
    def load_data(self):
        with open(self.output_dir+'/'+self.filename,"rb") as f:
            return pickle.load(f)

NameError: ignored

In [ ]:
class simMat_online_batcher:
    # 원래 논문(코드)에서는 batch의 크기를 고정하고 대신 batch마다 시간축의 길이를 유동적으로 변할 수 있게 모델링했다.
    # 하지만 그렇게 하려면 모델의 구조를 바꿔야 하는걸... 시간 다르게는 나중에 구현하기
    def __init__(self, input_dir, output_dir):
        self.name = 'simMat_online_batcher'
        self.input_dir = input_dir
        self.output_dir = output_dir
        
        # load data
        if len(os.listdir(self.output_dir))==0: self.generate()
        else: print('load exist data')
        self.filename = os.listdir(self.output_dir)[0]
        self.num_speaker = int(self.filename.split('_')[1]) # num_speaker는 이 batcher에서 의미없다. 일단 모델을 만들기 위해 설정
        self.specs_of_speakers = self.load_data()
    

    # simMat batcher를 online으로 하기 위해 각 화자마다 스펙트로그램 300개씩 뽑아 업로드
    # 사실상 naive_batcher가 만든 데이터 npy파일이 화자별로 정렬되어있는 것 뿐이다.
    def generate(self, num_frame=NUM_FRAME):

        specs_of_speaker = []
        for speaker in os.listdir(input_dir):
            X_speaker = []
            for utt in os.listdir(input_dir+'/'+speaker):
                spec = np.load(input_dir+'/'+speaker+'/'+utt)
                for i in range(0,spec.shape[0]-NUM_FRAME+1,NUM_FRAME): X_speaker.append(spec[i:i+NUM_FRAME,:])
                if len(X_speaker)>=NUM_PER_SPEAKER: break
            if len(X_speaker)<NUM_PER_SPEAKER: continue
            specs_of_speaker.append(np.array(X_speaker[:NUM_PER_SPEAKER]))
        specs_of_speaker = np.array(specs_of_speaker)
        num_speaker = len(specs_of_speaker)
        print('shape:', specs_of_speaker.shape)

        # save
        with open(output_dir+'/'+f"specsOfSpeakers_{num_speaker}_{NUM_PER_SPEAKER}_{NUM_FRAME}_{NUM_FBANK}.npy",'wb') as f:
            pickle.dump(specs_of_speaker,f)

    

    # num_speaker_per_batch : 배치당 화자 수. 논문=64 코드=4. 일단은 5개 발화로 등록한다 가정하고 5로 설정.
    # num_utt_per_speaker : 배치당 화자마다 발화 수. 논문=10 코드=5
    def get_batch(self, num_speaker_per_batch=5, num_utt_per_speaker=5):

        batch = []
        speakers = np.random.choice(self.num_speaker, num_speaker_per_batch, replace=False)
        for speaker in speakers:
            utts_idx = np.random.choice(NUM_PER_SPEAKER, num_utt_per_speaker, replace=False)
            batch.append(self.specs_of_speakers[speaker][utts_idx])
        batch = np.concatenate(batch)

        Y = np.zeros(batch.shape[0]) # dummy

        return batch,Y
    
    def train_generator(self):
        while True:
            yield self.get_batch()
        
    def load_data(self):
        with open(self.output_dir+'/'+self.filename,"rb") as f:
            return pickle.load(f)

In [ ]:
def get_batcher(batcher_name, input_dir, output_dir):
    if batcher_name=='naive_batcher': return naive_batcher(input_dir, output_dir)
    elif batcher_name=='simMat_batcher': return simMat_batcher(input_dir, output_dir)
    elif batcher_name=='simMat_online_batcher': return simMat_online_batcher(input_dir, output_dir)
    
    

In [ ]:
#simMat_batcher('C:/Users/LeeJunghun/Desktop/lingometer/data/NIKL_DIALOGUE_2020_PCM_v1.2_part1/NIKL_DIALOGUE_2020_PCM_v1.2_part1/npy/traintest','C:/Users/LeeJunghun/Desktop/lingometer/data/NIKL_DIALOGUE_2020_PCM_v1.2_part1/NIKL_DIALOGUE_2020_PCM_v1.2_part1/simMat_batcher')

number of speakers who has more than minimun: 167
total generated shape: (1000, 25, 128, 128, 1)
dummy Y shape: (1000, 25)
